In [18]:
import sys
import numpy as np
sys.path.insert(0, '../model_functions')
import primary_model as pm
import importlib
importlib.reload(pm)

<module 'primary_model' from '../model_functions/primary_model.py'>

In [2]:
data_path = "../../data/cull%i/model_data/" % 2

devtest_aa_dict = np.load(data_path + 'devtest_aa_dict.npy')[()]
devtest_cmap_dict = np.load(data_path + 'devtest_cmap_dict.npy')[()]
train_aa_dict = np.load(data_path + 'train_aa_dict.npy')[()]
train_cmap_dict = np.load(data_path + 'train_cmap_dict.npy')[()]
valid_aa_dict = np.load(data_path + 'valid_aa_dict.npy')[()]
valid_cmap_dict = np.load(data_path + 'valid_cmap_dict.npy')[()]

In [3]:
# def aa_generator(x, y):
#     """
#     Generator for feeding a single instance of an 
#     input and an output.
#     The generator is reset when all elements are used.

#     :param: input
#     :type:  dict
#     :param: label
#     :type:  dict
#     :returns: a single instance of input and label
#     :rtype:   (numpy array, numpy array)
#     """

#     keys = set(x.keys())

#     while True:
#         try:
#             key = random.sample(keys, 1)[0]
#             keys.remove(key)

#             one_hot_aa = x[key]
#             one_hot_aa = np.reshape(
#                 one_hot_aa, (1,) + one_hot_aa.shape
#             )
#             cmap = y[key]
#             cmap = np.reshape(cmap, (1,) + cmap.shape + (1,))
#             yield one_hot_aa, cmap

#         except ValueError:
#             # if out of keys, reinsert back the keys
#             keys = set(x.keys())

In [43]:
len (valid_aa_dict)
len (devtest_aa_dict)

106

In [3]:
import random


def aa_generator_batch(x, y, batch_size):
    """
    A generator for batches of sorted size.

    :param x: dictionary mapping keys to aa 1 hot
    :type  x: dict
    :param y: dictionary mapping keys to cmaps
    :type  y: dict
    :param batch_size: size of batch
    :type  batch_size: int
    :returns: aa batch, cmap batch
    :rtype:   (np.array, np.array)
    """

    aa = [(pdb_id, array) for (pdb_id, array) in x.items()]
    aa.sort(
        key=lambda x: x[1].shape[0],
        reverse=True
    )
    cmaps = [(pdb_id, array) for (pdb_id, array) in y.items()]
    cmaps.sort(
        key=lambda x: x[1].shape[0],
        reverse=True
    )

    def check_if_aligned(x, y):
        """
        Check if the sorting is aligned.

        :param x: list of tuples (a, c)
        :type  x: list
        :param y: list of tuples (a, b)
        :type  y: list
        """

        if len(x) != len(y):
            raise ValueError("Lengths do not match.")

        for i in range(len(x)):
            if (x[i][0] != y[i][0]):
                raise ValueError("Not sorted correctly")

    check_if_aligned(aa, cmaps)

    def create_batches(aa, cmaps, batch_size):
        """
        Create the batches.

        :param aa: list of tuple (pdb_id, aa 1 hot)
        :type  aa: list
        :param cmaps: list of tuple (pdb_id, cmap)
        :type  cmaps: list
        :param batch_size: size of batch
        :type  batch_size: int
        :returns: dictionary mapping a key to a tuple
                  (aa batch, cmap batch)
        :rtype:   dict
        """

        all_batches = {}
        for i in range(0, len(aa), batch_size):
            aa_batch = aa[i:i+batch_size]
            cmap_batch = cmaps[i:i+batch_size]
            max_length = aa[i][1].shape[0]

            aa_batch = [
                np.pad(
                    array,
                    ((0, max_length - array.shape[0]),
                     (0, 0)),
                    'constant')
                for (pdb_id, array) in aa_batch
            ]

            cmap_batch = [
                np.pad(
                    array,
                    ((0, max_length - array.shape[0]),
                     (0, max_length - array.shape[1])),
                    'constant')
                for (pdb_id, array) in cmap_batch
            ]

            cmap_batch = [
                np.reshape(
                    batch, batch.shape + (1,)) 
                for batch in cmap_batch
            ]
            
                
            stacked_aa_batch = np.stack(aa_batch, axis=0)
            stacked_cmap_batch = np.stack(cmap_batch, axis=0)

            all_batches["batch" +
                        str(i)] = (stacked_aa_batch, stacked_cmap_batch)

        return all_batches

    all_batches = create_batches(aa, cmaps, batch_size)
    keys = set(all_batches.keys())

    while True:
        try:
            key = random.sample(keys, 1)[0]
            keys.remove(key)
            aa_batch, cmap_batch = all_batches[key]

            yield aa_batch, cmap_batch

        except ValueError:
            # if out of keys, reinsert back the keys
            keys = set(all_batches.keys())

In [106]:
# i = 0

# for aa_bro, cmap_bro in aa_generator_batch(train_aa_dict, train_cmap_dict, 50):
#     if i == 50:
#         break
        
#     print(aa_bro.shape, cmap_bro.shape)
#     i +=1

GGGGG
(50, 323, 20) (50, 323, 323, 1)
(50, 117, 20) (50, 117, 117, 1)
(46, 67, 20) (46, 67, 67, 1)
(50, 155, 20) (50, 155, 155, 1)
(50, 140, 20) (50, 140, 140, 1)
(50, 442, 20) (50, 442, 442, 1)
(50, 233, 20) (50, 233, 233, 1)
(50, 373, 20) (50, 373, 373, 1)
(50, 128, 20) (50, 128, 128, 1)
(50, 178, 20) (50, 178, 178, 1)
(50, 85, 20) (50, 85, 85, 1)
(50, 102, 20) (50, 102, 102, 1)
(50, 203, 20) (50, 203, 203, 1)
(50, 277, 20) (50, 277, 277, 1)
(50, 678, 20) (50, 678, 678, 1)
(50, 323, 20) (50, 323, 323, 1)
(50, 140, 20) (50, 140, 140, 1)
(50, 277, 20) (50, 277, 277, 1)
(50, 128, 20) (50, 128, 128, 1)
(50, 442, 20) (50, 442, 442, 1)
(50, 203, 20) (50, 203, 203, 1)
(50, 373, 20) (50, 373, 373, 1)
(50, 85, 20) (50, 85, 85, 1)
(50, 155, 20) (50, 155, 155, 1)
(46, 67, 20) (46, 67, 67, 1)
(50, 117, 20) (50, 117, 117, 1)
(50, 102, 20) (50, 102, 102, 1)
(50, 678, 20) (50, 678, 678, 1)
(50, 233, 20) (50, 233, 233, 1)
(50, 178, 20) (50, 178, 178, 1)
(50, 678, 20) (50, 678, 678, 1)
(50, 117, 20) 

In [74]:
# (aa_test[0])
aa_test.shape

(46, 67, 20)

In [42]:
# train_aa_dict['1a1x'].shape
len(train_aa_dict)

746

In [32]:
test1model = pm.create_architecture(3, 6) = np.linspace(1,20,20).reshape(4,5)
test3 = np.linspace(21,40,20).reshape(4,5)
test2 = np.linspace(1,30,30).reshape(6,5)
test1

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15.],
       [16., 17., 18., 19., 20.]])

In [95]:
test2

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15.],
       [16., 17., 18., 19., 20.],
       [21., 22., 23., 24., 25.],
       [26., 27., 28., 29., 30.]])

In [99]:
# test2
test2b = np.reshape(test2, test2.shape + (1,))
test2b#.shape

array([[[ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.]],

       [[ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.]],

       [[11.],
        [12.],
        [13.],
        [14.],
        [15.]],

       [[16.],
        [17.],
        [18.],
        [19.],
        [20.]],

       [[21.],
        [22.],
        [23.],
        [24.],
        [25.]],

       [[26.],
        [27.],
        [28.],
        [29.],
        [30.]]])

In [33]:
test3

array([[21., 22., 23., 24., 25.],
       [26., 27., 28., 29., 30.],
       [31., 32., 33., 34., 35.],
       [36., 37., 38., 39., 40.]])

In [54]:
max_height = 10
test3a = np.pad(test3, ((0, 10 - len(test3)), (0, 0)), 'constant')
test2a = np.pad(test2, ((0, 10 - len(test2)), (0, 0)), 'constant')
test4 = np.stack((test3a, test2a), axis=0)

In [62]:
# len(test3)
test3.shape

(4, 5)

In [55]:
test4#.shape

array([[[21., 22., 23., 24., 25.],
        [26., 27., 28., 29., 30.],
        [31., 32., 33., 34., 35.],
        [36., 37., 38., 39., 40.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10.],
        [11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20.],
        [21., 22., 23., 24., 25.],
        [26., 27., 28., 29., 30.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]])

In [67]:
importlib.reload(pm)

<module 'primary_model' from '../model_functions/primary_model.py'>

In [68]:
model = pm.create_architecture(3, 4)

In [69]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, 20)     0                                            
__________________________________________________________________________________________________
1d_convnet_layer1a (Conv1D)     (None, None, 20)     6820        input_layer[0][0]                
__________________________________________________________________________________________________
1d_convnet_batch_norm1a (BatchN (None, None, 20)     80          1d_convnet_layer1a[0][0]         
__________________________________________________________________________________________________
1d_convnet_layer1b (Conv1D)     (None, None, 20)     6820        1d_convnet_batch_norm1a[0][0]    
__________________________________________________________________________________________________
1d_convnet

In [70]:
model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        sample_weight_mode="temporal",
        metrics=['accuracy']
    )

In [66]:
%%time
history = model.fit_generator(
        pm.aa_generator(train_aa_dict, train_cmap_dict),
        validation_data=pm.aa_generator(valid_aa_dict, valid_cmap_dict),
        steps_per_epoch=6, 
        epochs=1,
        validation_steps=1,
        callbacks=pm.callbacks_list
    )

/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [8]:
%%time
history = model.fit_generator(
        aa_generator_batch(train_aa_dict, train_cmap_dict, 15),
#         pm.aa_generator(train_aa_dict, train_cmap_dict),
#         validation_data=pm.aa_generator(valid_aa_dict, valid_cmap_dict),
        steps_per_epoch=2, 
        epochs=1,
        validation_steps=1,
        callbacks=pm.callbacks_list
    )

/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
2/2 [==============================] - 101s 51s/step - loss: 4.5038 - acc: 0.6361
CPU times: user 3min 23s, sys: 43.8 s, total: 4min 7s
Wall time: 1min 44s


In [9]:
len(train_aa_dict)
model.save('my_model.h5') 

In [25]:
aaa = (1, 2, 3)
aaa[:-1]

(1, 2)